In [ ]:
import localSettings as ls
from load_data_run123 import *
from unblinding_far_sideband import *
from matplotlib import pyplot as plt
from matplotlib import gridspec
from scipy.optimize import curve_fit

In [ ]:
def Eff(df,var,query,acceptance,bin_edges,absval=False):
    #print acceptance
    bin_centers = 0.5*(bin_edges[1:]+bin_edges[:-1])
    bins = []
    bin_eff = []
    bin_err = []
    for i in range(len(bin_centers)):
        binmin = bin_edges[i]
        binmax = bin_edges[i+1]
        bincut = '%s > %f and %s < %f'%(var,binmin,var,binmax)
        if (absval == True):
            bincut = '(%s > %f and %s < %f) or (%s > -%f and %s < -%f)'%(var,binmin,var,binmax,var,binmax,var,binmin)
        if (acceptance != ''): bincut += ' and %s'%acceptance
        #print bincut
        df_tmp =  df.query(bincut) # cut on bin range for desired var.
        df_sub = df_tmp.query(query) # apply constrain 
        if (df_tmp.shape[0] == 0): continue
        eff = df_sub.shape[0] / float( df_tmp.shape[0] )
        err = np.sqrt( eff*(1-eff)/df_tmp.shape[0] )
        bin_eff.append( eff )
        bin_err.append( err )
        bins.append(bin_centers[i])
        #print 'eff = %.02f @ bin = %.02f'%(eff,bin_centers[i])
    return np.array(bins),np.array(bin_eff),np.array(bin_err)

In [ ]:
def _ratio_err(num, den, num_err, den_err):
    n, d, n_e, d_e = num, den, num_err, den_err
    n[n == 0] = 0.00001
    return np.array([
        np.sqrt( ( n_e[i] / d[i] )**2 + ( n[i] * d_e[i] / (d[i]*d[i]) )**2) if d[i]>0 else 0
        for i, k in enumerate(num)
    ])

In [ ]:
# Define model function to be used to fit to the data above:
def gauss(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))

In [ ]:
ntuple_path = ls.ntuple_path+"../1102/unfiltered/"
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

fcc1 = "prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run1_reco2"
fcc3 = "prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run3_G_reco2"

fnc1 = "prodgenie_nc_pi0_uboone_overlay-v08_00_00_26_run1_reco2_reco2"
fnc3 = "prodgenie_nc_pi0_uboone_overlay_mcc9.1_v08_00_00_26_run3_G_reco2"

ucc1 = uproot.open(ntuple_path+fcc1+".root")[fold][tree]
unc1 = uproot.open(ntuple_path+fnc1+".root")[fold][tree]
ucc3 = uproot.open(ntuple_path+fcc3+".root")[fold][tree]
unc3 = uproot.open(ntuple_path+fnc3+".root")[fold][tree]

vardict = get_variables()
variables = vardict['VARIABLES']+vardict['NUEVARS']+vardict['PI0VARS']+['weightSplineTimesTune']+['pi0_mcgamma0_pz','pi0_mcgamma1_pz','pi0_mcgamma0_e','pi0_mcgamma1_e']
variables = list(set(variables))

ccr1 = ucc1.pandas.df(variables, flatten=False)
ncr1 = unc1.pandas.df(variables, flatten=False)
ccr3 = ucc3.pandas.df(variables, flatten=False)
ncr3 = unc3.pandas.df(variables, flatten=False)


In [ ]:
uproot_v = [ucc1,ucc3,unc1,unc3]
df_v = [ccr1,ccr3,ncr1,ncr3]

In [ ]:
for i,df in enumerate(df_v):
    up = uproot_v[i]
    process_uproot(up,df)
    process_uproot_eta(up,df)
    process_uproot_ccncpi0vars(up,df)
    #process_uproot_recoveryvars(up,df)

for i,df in enumerate(df_v):
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    # and the 2d angle difference
    df['anglediff_Y'] = np.abs(df['secondshower_Y_dir']-df['shrclusdir2'])

for i,df in enumerate(df_v):
    df['shr_tkfit_nhits_tot'] = (df['shr_tkfit_nhits_Y']+df['shr_tkfit_nhits_U']+df['shr_tkfit_nhits_V'])
    df.loc[:,'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_Y']
    df.loc[(df['shr_tkfit_nhits_U']>df['shr_tkfit_nhits_Y']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_U']
    df.loc[(df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_Y']) & (df['shr_tkfit_nhits_V']>df['shr_tkfit_nhits_U']),'shr_tkfit_dedx_max'] = df['shr_tkfit_dedx_V']

INTERCEPT = 0.0
SLOPE = 0.83
    
for i,df in enumerate(df_v):
    df['asymm'] = np.abs(df['pi0_energy1_Y']-df['pi0_energy2_Y'])/(df['pi0_energy1_Y']+df['pi0_energy2_Y'])
    df['pi0energy'] = 134.98 * np.sqrt( 2. / ( (1-(df['asymm'])**2) * (1-df['pi0_gammadot']) ) )
    df['pi0momentum'] = np.sqrt(df['pi0energy']**2 - 134.98**2)
    df['pi0beta'] = df['pi0momentum']/df['pi0energy']
    df['pi0thetacm'] = df['asymm']/df['pi0beta']
    df['pi0momx'] = df['pi0_energy2_Y']*df['pi0_dir2_x'] + df['pi0_energy1_Y']*df['pi0_dir1_x']
    df['pi0momy'] = df['pi0_energy2_Y']*df['pi0_dir2_y'] + df['pi0_energy1_Y']*df['pi0_dir1_y']
    df['pi0momz'] = df['pi0_energy2_Y']*df['pi0_dir2_z'] + df['pi0_energy1_Y']*df['pi0_dir1_z']
    df['pi0energyraw'] = df['pi0_energy2_Y'] + df['pi0_energy1_Y']
    df['pi0energyraw_corr'] = df['pi0energyraw'] / SLOPE
    df['pi0momanglecos'] = df['pi0momz'] / df['pi0energyraw']
    df['epicospi'] = df['pi0energy'] * (1-df['pi0momanglecos'])
    df['boost'] = (np.abs(df['pi0_energy1_Y']-df['pi0_energy2_Y'])/SLOPE)/(np.sqrt((df['pi0energy'])**2-134.98**2))
    df['pi0_mass_Y_corr'] = df['pi0_mass_Y']/SLOPE
    df['pi0energymin'] = 134.98 * np.sqrt( 2. / (1-df['pi0_gammadot']) )
    df['pi0energyminratio'] = df['pi0energyraw_corr'] / df['pi0energymin']

# define some energy-related variables
for i,df in enumerate(df_v):
    df["reco_e"] = (df["shr_energy_tot_cali"] + INTERCEPT) / SLOPE + df["trk_energy_tot"]

# and a way to filter out data
for i,df in enumerate(df_v):
    df["bnbdata"] = np.zeros_like(df["shr_energy"])
    df["extdata"] = np.zeros_like(df["shr_energy"])

for i,df in enumerate(df_v):
    df["pi0energygev"] = df["pi0energy"]*0.001

#TRAINVAR = ["shr_score","tksh_distance","tksh_angle",
#            "shr_tkfit_dedx_max",
#            "trkfit","trkpid",
#            "subcluster","shrmoliereavg",
#            "trkshrhitdist2","hits_ratio",
#            "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","anglediff_Y",
#            "CosmicIPAll3D","CosmicDirAll3D"]
#
#LABELS =  ['pi0','nonpi0']
#    
#for label, bkg_query in zip(LABELS, nue_booster.bkg_queries):
#    with open(ls.pickle_path+'booster_%s_0304_extnumi.pickle' % label, 'rb') as booster_file:
#        booster = pickle.load(booster_file)
#        for df in df_v:
#            df[label+"_score"] = booster.predict(xgb.DMatrix(df[TRAINVAR]),ntree_limit=booster.best_iteration)

In [ ]:
ccr1['POT'] = np.ones_like(ccr1['nslice'])*3.48E+21
ncr1['POT'] = np.ones_like(ncr1['nslice'])*2.66E+21
ccr3['POT'] = np.ones_like(ccr3['nslice'])*6.43E+21
ncr3['POT'] = np.ones_like(ncr3['nslice'])*2.31E+21
#cc = ccr1
#nc = ncr1
pi0s = pd.concat([ccr1,ccr3,ncr1,ncr3],ignore_index=True)

In [ ]:

CCNCSEL = "n_showers_contained >= 2 and nslice == 1 and contained_fraction > 0.4 and shr_energy_tot_cali > 0.07 and ( (_opfilter_pe_beam > 0 and _opfilter_pe_veto < 20) or bnbdata == 1 or extdata == 1) and CosmicIPAll3D > 30. and shr_trk_sce_end_y < 105 and hits_y > 80 and shr_score < 0.25 and topological_score > 0.1"
CCSEL = CCNCSEL +  ' and ((trkpid>0.6 and n_tracks_contained>0 and n_tracks_contained == n_tracks_tot) or (n_tracks_contained < n_tracks_tot))'
NCSEL = CCNCSEL + ' and (trkpid<0.6 or n_tracks_contained==0) and (n_tracks_contained == n_tracks_tot)'


In [ ]:
#bin_edges = np.linspace(0,2.5,6)
#VAR = 'nu_e'
#LABEL = 'True Neutrino Energy [GeV]'

#bin_edges_e = np.array([0.1, 0.25, 0.4, 0.7, 1.0, 1.5])
bin_edges_e = np.array([0.1, 0.25, 0.4, 0.7, 1.5])

#bin_edges_t = np.array([-1, -0.6, -0.2, 0.1, 0.4, 0.6, 0.8, 1.0])
#bin_edges_t = np.array([-1, -0.5, 0., 0.4, 0.7, 1.0])
bin_edges_t = np.array([-1, -0.4, 0.2, 0.7, 1.0])

if True:
    VART = 'pi0_e'
    LABELT = 'True Pi0 Energy [GeV]'
    VARR = 'pi0energygev'
    LABELR = 'Reco Pi0 Energy [GeV]'
    bin_edges = bin_edges_e

if False:
    VART = 'leadPi0_uz'
    LABELT = 'True Pi0 Cos Theta'
    VARR = 'pi0momanglecos'
    LABELR = 'Reco Pi0 Cos Theta'
    bin_edges = bin_edges_t

print(bin_edges)

ACCEPTANCE = 'isVtxInFiducial == 1 and truthFiducial == 1 and npi0==1 and (category == 21 or category == 31)'

In [ ]:
plt.figure(figsize=(7,6))

ax1=plt.subplot(1, 1, 1)

QUERY = CCNCSEL

lables = ['cc','nc']
cuts = ['ccnc==0','ccnc==1']
colors = ['b','r']

effs = []

for i,q in enumerate(cuts):
    fmt = colors[i]+'o-'
    centers1,vals1,errs1 = Eff(pi0s,VART,QUERY,ACCEPTANCE+' and '+q,bin_edges)
    ax1.errorbar(centers1,vals1,yerr=errs1,xerr=0.5*(bin_edges[1:]-bin_edges[:-1]),fmt=fmt,label=lables[i])
    effs.append(vals1)

ax1.set_xlabel(LABELT)
ax1.set_ylabel('CCNC Pi0 Selection Efficiency')
ax1.set_ylim(0.,0.5)
ax1.set_xlim(bin_edges[0],bin_edges[-1])
ax1.legend(loc=2)
ax1.grid(True)

plt.show()

effCC = effs[0]
effNC = effs[1]
print('effCC =',effCC)
print('effNC =',effNC)

In [ ]:
plt.figure(figsize=(14,6))

ax1=plt.subplot(1, 2, 1)
ax2=plt.subplot(1, 2, 2)


QUERY  = ACCEPTANCE+' and '+CCNCSEL
QUERY1 = ACCEPTANCE+' and '+CCSEL
QUERY2 = ACCEPTANCE+' and '+NCSEL

print(QUERY)
print(QUERY1)
print(QUERY2)

centers1,vals1,errs1 = Eff(pi0s,VART,QUERY1+' and ccnc==0',QUERY+' and ccnc==0',bin_edges)
ax1.errorbar(centers1,vals1,yerr=errs1,xerr=0.5*(bin_edges[1:]-bin_edges[:-1]),fmt='k',label='CCSEL')
fracCC = vals1
centers1,vals1,errs1 = Eff(pi0s,VART,QUERY2+' and ccnc==0',QUERY+' and ccnc==0',bin_edges)
ax1.errorbar(centers1,vals1,yerr=errs1,xerr=0.5*(bin_edges[1:]-bin_edges[:-1]),fmt='g',label='NCSEL')

centers2,vals2,errs2 = Eff(pi0s,VART,QUERY1+' and ccnc==1',QUERY+' and ccnc==1',bin_edges)
ax2.errorbar(centers2,vals2,yerr=errs2,xerr=0.5*(bin_edges[1:]-bin_edges[:-1]),fmt='k',label='CCSEL')
centers2,vals2,errs2 = Eff(pi0s,VART,QUERY2+' and ccnc==1',QUERY+' and ccnc==1',bin_edges)
ax2.errorbar(centers2,vals2,yerr=errs2,xerr=0.5*(bin_edges[1:]-bin_edges[:-1]),fmt='g',label='NCSEL')
fracNC = vals2

ax1.set_xlabel(LABELT)
ax2.set_xlabel(LABELT)
ax1.set_ylabel('Fraction of CC Pi0 Events')
ax2.set_ylabel('Fraction of NC Pi0 Events')
#ax1.set_xticks(bin_edges)
#ax2.set_xticks(bin_edges)
ax1.set_ylim(0.,1.0)
ax2.set_ylim(0.,1.0)
ax1.set_xlim(bin_edges[0],bin_edges[-1])
ax2.set_xlim(bin_edges[0],bin_edges[-1])
ax1.legend(loc=2)
ax2.legend(loc=2)
ax1.grid(True)
ax2.grid(True)

plt.show()

print('fracCC =',fracCC)
print('fracNC =',fracNC)

In [ ]:
plt.figure(figsize=(14,6))

ax1=plt.subplot(1, 2, 1)
ax2=plt.subplot(1, 2, 2)

QUERY1 = CCSEL
QUERY2 = NCSEL

lables = ['cc','nc']
cuts = ['ccnc==0','ccnc==1']
colors = ['b','r']

for i,q in enumerate(cuts):
    fmt = colors[i]+'o-'
    centers1,vals1,errs1 = Eff(pi0s,VART,QUERY1,ACCEPTANCE+' and '+q,bin_edges)
    ax1.errorbar(centers1,vals1,yerr=errs1,xerr=0.5*(bin_edges[1:]-bin_edges[:-1]),fmt=fmt,label=lables[i])
    centers2,vals2,errs2 = Eff(pi0s,VART,QUERY2,ACCEPTANCE+' and '+q,bin_edges)
    ax2.errorbar(centers2,vals2,yerr=errs2,xerr=0.5*(bin_edges[1:]-bin_edges[:-1]),fmt=fmt,label=lables[i])

ax1.set_xlabel(LABELT)
ax2.set_xlabel(LABELT)
ax1.set_ylabel('CC Pi0 Selection Efficiency')
ax2.set_ylabel('NC Pi0 Selection Efficiency')
#ax1.set_xticks(bin_edges)
#ax2.set_xticks(bin_edges)
ax1.set_ylim(0.,0.5)
ax2.set_ylim(0.,0.5)
ax1.set_xlim(bin_edges[0],bin_edges[-1])
ax2.set_xlim(bin_edges[0],bin_edges[-1])
ax1.legend(loc=2)
ax2.legend(loc=2)
ax1.grid(True)
ax2.grid(True)

plt.show()


In [ ]:
plt.figure(figsize=(6, 6))
gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])
ax1 = plt.subplot(gs[0])
myCCSEL = ACCEPTANCE+" and ccnc==0"
myNCSEL = ACCEPTANCE+" and ccnc==1"
varCC = pi0s.query(myCCSEL)[VART]
varNC = pi0s.query(myNCSEL)[VART]
wghCC = 6.95e20*pi0s.query(myCCSEL)['weightSplineTimesTune']/pi0s.query(myCCSEL)['POT']
wghNC = 6.95e20*pi0s.query(myNCSEL)['weightSplineTimesTune']/pi0s.query(myNCSEL)['POT']
n, b, p = plt.hist([varCC,varNC], bins=bin_edges,
                   weights=[wghCC,wghNC],
                   histtype='step',linestyle='solid',color=('blue','red'), linewidth=1.2,
                   stacked=False, label=('true CC','true NC'))
plt.title('truth, in FV')
ax1.set_ylabel('selected events (6.95e20 POT)')
ax1.legend(loc=1)
#ax1.set_ylim(0., 300)
ax1.set_xticks([])
#
ax2 = plt.subplot(gs[1])
ax2.set_xlabel(LABELT)
ax2.set_ylabel('NC/CC')
bincenters = 0.5 * (b[1:] + b[:-1])
bin_size = [(b[i + 1] - b[i]) / 2 for i in range(len(b) - 1)]
ax2.errorbar(bincenters, n[1] / n[0], xerr=bin_size, yerr=_ratio_err(n[1],n[0],np.sqrt(n[1]),np.sqrt(n[0])), fmt="rs")#ratio_error
ax2.set_ylim(0.5, 1.5)
ax2.grid(True)
print(n)

trueratio = n[1] / n[0]

print("trueratio =", trueratio)


In [ ]:
plt.figure(figsize=(6, 6))
gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])
ax1 = plt.subplot(gs[0])
myCCSEL = ACCEPTANCE+' and '+CCNCSEL+" and ccnc==0"
myNCSEL = ACCEPTANCE+' and '+CCNCSEL+" and ccnc==1"
varCC = pi0s.query(myCCSEL)[VART]
varNC = pi0s.query(myNCSEL)[VART]
wghCC = 6.95e20*pi0s.query(myCCSEL)['weightSplineTimesTune']/pi0s.query(myCCSEL)['POT']
wghNC = 6.95e20*pi0s.query(myNCSEL)['weightSplineTimesTune']/pi0s.query(myNCSEL)['POT']
n, b, p = plt.hist([varCC,varNC], bins=bin_edges,
                   weights=[wghCC,wghNC],
                   histtype='step',linestyle='solid',color=('blue','red'), linewidth=1.2, 
                   stacked=False, label=('true CC after CCNCSEL','true NC after CCNCSEL'))
plt.title('truth, after selection')
ax1.set_ylabel('selected events (6.95e20 POT)')
ax1.legend(loc=1)
#ax1.set_ylim(0., 300)
ax1.set_xticks([])
#
ax2 = plt.subplot(gs[1])
ax2.set_xlabel(LABELT)
ax2.set_ylabel('NC/CC')
bincenters = 0.5 * (b[1:] + b[:-1])
bin_size = [(b[i + 1] - b[i]) / 2 for i in range(len(b) - 1)]
ax2.errorbar(bincenters, n[1] / n[0], xerr=bin_size, yerr=_ratio_err(n[1],n[0],np.sqrt(n[1]),np.sqrt(n[0])), fmt="rs")#ratio_error
ax2.set_ylim(0.5, 1.5)
ax2.grid(True)
print(np.sum(n,axis=1))
print(n)

In [ ]:
plt.figure(figsize=(6, 6))
gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])
ax1 = plt.subplot(gs[0])
myCCSEL = ACCEPTANCE+' and '+CCSEL+" and ccnc==0"
myNCSEL = ACCEPTANCE+' and '+NCSEL+" and ccnc==1"
varCC = pi0s.query(myCCSEL)[VART]
varNC = pi0s.query(myNCSEL)[VART]
wghCC = 6.95e20*pi0s.query(myCCSEL)['weightSplineTimesTune']/pi0s.query(myCCSEL)['POT']
wghNC = 6.95e20*pi0s.query(myNCSEL)['weightSplineTimesTune']/pi0s.query(myNCSEL)['POT']
n, b, p = plt.hist([varCC,varNC], bins=bin_edges,
                   weights=[wghCC,wghNC],
                   histtype='step',linestyle='solid',color=('blue','red'), linewidth=1.2, 
                   stacked=False, label=('true CC after CCSEL','true NC after NCSEL'))
plt.title('truth, after selection')
ax1.set_ylabel('selected events (6.95e20 POT)')
ax1.legend(loc=1)
#ax1.set_ylim(0., 300)
ax1.set_xticks([])
#
ax2 = plt.subplot(gs[1])
ax2.set_xlabel(LABELT)
ax2.set_ylabel('NC/CC')
bincenters = 0.5 * (b[1:] + b[:-1])
bin_size = [(b[i + 1] - b[i]) / 2 for i in range(len(b) - 1)]
ax2.errorbar(bincenters, n[1] / n[0], xerr=bin_size, yerr=_ratio_err(n[1],n[0],np.sqrt(n[1]),np.sqrt(n[0])), fmt="rs")#ratio_error
ax2.set_ylim(0.5, 1.5)
ax2.grid(True)
print(np.sum(n,axis=1))
print(n)

In [ ]:
plt.figure(figsize=(6, 6))
#gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])
#ax1 = plt.subplot(gs[0])
myCCSEL = ACCEPTANCE+' and '+CCSEL
myNCSEL = ACCEPTANCE+' and '+NCSEL
varCC = pi0s.query(myCCSEL)[VART]
varNC = pi0s.query(myNCSEL)[VART]
wghCC = 6.95e20*pi0s.query(myCCSEL)['weightSplineTimesTune']/pi0s.query(myCCSEL)['POT']
wghNC = 6.95e20*pi0s.query(myNCSEL)['weightSplineTimesTune']/pi0s.query(myNCSEL)['POT']
n, b, p = plt.hist([varCC,varNC], bins=bin_edges,
                   weights=[wghCC,wghNC],
                   histtype='step',linestyle='solid',color=('blue','red'), linewidth=1.2, 
                   stacked=False, label=('true pi0 after CCSEL','true pi0 after NCSEL'))
plt.title('truth, after selection')
plt.ylabel('selected events (6.95e20 POT)')
plt.xlabel(LABELT)
plt.legend(loc=1)
#ax1.set_ylim(0., 300)

truetrueCCsel = n[0]
truetrueNCsel = n[1]

print(np.sum(n,axis=1))
print('truetrueCCsel:',truetrueCCsel)
print('truetrueNCsel:',truetrueNCsel)


In [ ]:
plt.figure(figsize=(6, 6))
#gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])
#ax1 = plt.subplot(gs[0])
myCCSEL = ACCEPTANCE+' and '+CCSEL
myNCSEL = ACCEPTANCE+' and '+NCSEL
varCC = pi0s.query(myCCSEL)[VARR]
varNC = pi0s.query(myNCSEL)[VARR]
wghCC = 6.95e20*pi0s.query(myCCSEL)['weightSplineTimesTune']/pi0s.query(myCCSEL)['POT']
wghNC = 6.95e20*pi0s.query(myNCSEL)['weightSplineTimesTune']/pi0s.query(myNCSEL)['POT']
n, b, p = plt.hist([varCC,varNC], bins=bin_edges,
                   weights=[wghCC,wghNC],
                   histtype='step',linestyle='solid',color=('blue','red'), linewidth=1.2, 
                   stacked=False, label=('true pi0 after CCSEL','true pi0 after NCSEL'))
plt.title('truth, after selection')
plt.ylabel('selected events (6.95e20 POT)')
plt.legend(loc=2)
plt.xlabel(LABELR)
#ax1.set_ylim(0., 300)

recoCCsel = n[0]
recoNCsel = n[1]

print(np.sum(n,axis=1))
print('recoCCsel:',recoCCsel)
print('recoNCsel:',recoNCsel)


In [ ]:
mySEL = ACCEPTANCE+' and '+CCNCSEL
#mySEL = ACCEPTANCE+' and '+NCSEL+" and category == 21"
#mySEL = ACCEPTANCE+' and '+NCSEL+" and category == 31"
#mySEL = ACCEPTANCE+' and '+CCSEL+" and category == 21"
#mySEL = ACCEPTANCE+' and '+CCSEL+" and category == 31"
#mySEL = ACCEPTANCE+' and '+CCNCSEL+" and category == 21"
#mySEL = ACCEPTANCE+' and '+CCNCSEL+" and category == 31"
#mySEL = ACCEPTANCE+' and '+CCSEL
#mySEL = ACCEPTANCE+' and '+NCSEL

#find resolutions
plt.figure(figsize=(12, 6))

ax1 = plt.subplot(1,2,1)
var = pi0s.query(mySEL)["pi0energygev"]-pi0s.query(mySEL)["pi0_e"]
wgh = 6.95e20*pi0s.query(mySEL)['weightSplineTimesTune']/pi0s.query(mySEL)['POT']
n, b, p = plt.hist([var], bins=16, range=(-1,1),
                   weights=[wgh],
                   histtype='step',linestyle='solid',color=('black'), linewidth=1.2, 
                   stacked=False, label=('true pi0 after CCNCSEL'))
plt.title('truth, after selection')
ax1.set_ylabel('selected events (6.95e20 POT)')
ax1.set_xlabel("Ereco-Etrue [GeV]")
ax1.set_ylim(0., ax1.get_ylim()[1]*1.2)
ax1.legend(loc=2)

#gaussian fit
bin_centres = (b[:-1] + b[1:])/2
# p0 is the initial guess for the fitting coefficients (A, mu and sigma above)
p0 = [250., 0., 0.1]
coeff, var_matrix = curve_fit(gauss, bin_centres, n, p0=p0)
errs = np.sqrt(np.diag(var_matrix))
curvebins = np.linspace(int(b[0]),int(b[-1]),200)
# Get the fitted curve
hist_fit = gauss(curvebins, *coeff)
plt.text(0.25,ax1.get_ylim()[-1]*0.95,"sigma=%.3f+/-%.3f"%(coeff[2],errs[2]))
plt.plot(curvebins, hist_fit, label='Fitted data')


ax2 = plt.subplot(1,2,2)
var = pi0s.query(mySEL)["pi0momanglecos"]-pi0s.query(mySEL)["leadPi0_uz"]
wgh = 6.95e20*pi0s.query(mySEL)['weightSplineTimesTune']/pi0s.query(mySEL)['POT']
n, b, p = plt.hist([var], bins=16, range=(-1,1),
                   weights=[wgh],
                   histtype='step',linestyle='solid',color=('black'), linewidth=1.2, 
                   stacked=False, label=('true pi0 after CCNCSEL'))
plt.title('truth, after selection')
ax2.set_ylabel('selected events (6.95e20 POT)')
ax2.set_xlabel("CosThetaReco-CosThetaRtrue")
ax2.set_ylim(0., ax2.get_ylim()[1]*1.2)
ax2.legend(loc=2)

#gaussian fit
bin_centres = (b[:-1] + b[1:])/2
# p0 is the initial guess for the fitting coefficients (A, mu and sigma above)
p0 = [250., 0., 0.1]
coeff, var_matrix = curve_fit(gauss, bin_centres, n, p0=p0)
errs = np.sqrt(np.diag(var_matrix))
curvebins = np.linspace(int(b[0]),int(b[-1]),200)
# Get the fitted curve
hist_fit = gauss(curvebins, *coeff)
plt.text(0.25,ax2.get_ylim()[-1]*0.95,"sigma=%.2f+/-%.2f"%(coeff[2],errs[2]))
plt.plot(curvebins, hist_fit, label='Fitted data')

plt.show()


In [ ]:
plt.figure(figsize=(14, 4))
plt.subplot(121)
x = pi0s.query(mySEL)["pi0energygev"]
y = pi0s.query(mySEL)["pi0energygev"]-pi0s.query(mySEL)["pi0_e"]
h = plt.hist2d(x, y, bins=[bin_edges_e,np.linspace(-1,1,16)])#, cmap=plt.cm.Greys)#, norm=LogNorm()
plt.xlabel('reco pi0 energy [GeV]')
plt.ylabel('reco-true pi0 energy [GeV]')
plt.colorbar()

plt.subplot(122)
x = pi0s.query(mySEL)["pi0momanglecos"]
y = pi0s.query(mySEL)["pi0momanglecos"]-pi0s.query(mySEL)["leadPi0_uz"]
plt.hist2d(x, y, bins=[bin_edges_t,np.linspace(-1,1,16)])#, cmap=plt.cm.Greys)#, norm=LogNorm()
plt.xlabel('reco pi0 cos theta')
plt.ylabel('reco-true pi0 cos theta')
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(14, 4))
plt.subplot(121)
x = pi0s.query(mySEL)["pi0_e"]
y = pi0s.query(mySEL)["pi0energygev"]
he2d = plt.hist2d(x, y, bins=bin_edges_e)
plt.xlabel('true pi0 energy [GeV]')
plt.ylabel('reco pi0 energy [GeV]')
plt.colorbar()

plt.subplot(122)
x = pi0s.query(mySEL)["leadPi0_uz"]
y = pi0s.query(mySEL)["pi0momanglecos"]
hc2d = plt.hist2d(x, y, bins=bin_edges_t)
plt.xlabel('true pi0 cos theta')
plt.ylabel('reco pi0 cos theta')
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))
plt.subplot(121)
a = he2d[0]
a = a / a.sum(axis=0)
plt.imshow(a,origin='lower')
for i in range(len(a[0])):
    for j in range(len(a[0])):
        text = plt.text(j, i, "%.2f"%a[i, j],ha="center", va="center", color="w")
plt.xlabel('reco pi0 energy bin')
plt.ylabel('true pi0 energy bin')
plt.colorbar()

plt.subplot(122)
b = hc2d[0]
b = b / b.sum(axis=0)
plt.imshow(b,origin='lower')
for i in range(len(b[0])):
    for j in range(len(b[0])):
        text = plt.text(j, i, "%.2f"%b[i, j],ha="center", va="center", color="w")
plt.xlabel('reco pi0 cos theta bin')
plt.ylabel('true pi0 cos theta bin')
plt.colorbar()

plt.show()

In [ ]:
myCCSEL = ACCEPTANCE+' and '+CCSEL
myNCSEL = ACCEPTANCE+' and '+NCSEL

fig = plt.figure(figsize=(14, 4))
plt.subplot(121)
x = pi0s.query(myCCSEL)[VART]
y = pi0s.query(myCCSEL)[VARR]
hCC2d = plt.hist2d(x, y, bins=bin_edges)
plt.xlabel(LABELT)
plt.ylabel(LABELR)
plt.colorbar()
plt.subplot(122)
x = pi0s.query(myNCSEL)[VART]
y = pi0s.query(myNCSEL)[VARR]
hNC2d = plt.hist2d(x, y, bins=bin_edges)
plt.xlabel(LABELT)
plt.ylabel(LABELR)
plt.colorbar()
plt.show()

plt.figure(figsize=(10, 4))
#
plt.subplot(121)
ccsm = hCC2d[0]
ccsm = ccsm / ccsm.sum(axis=0)
plt.imshow(ccsm,origin='lower')
for i in range(len(ccsm[0])):
    for j in range(len(ccsm[0])):
        text = plt.text(j, i, "%.2f"%ccsm[i, j],ha="center", va="center", color="w")
plt.title("CCpi0 selection")
plt.xlabel("true bin")
plt.ylabel("reco bin")
plt.colorbar()
#
plt.subplot(122)
ncsm = hNC2d[0]
ncsm = ncsm / ncsm.sum(axis=0)
plt.imshow(ncsm,origin='lower')
for i in range(len(ncsm[0])):
    for j in range(len(ncsm[0])):
        text = plt.text(j, i, "%.2f"%ncsm[i, j],ha="center", va="center", color="w")
plt.title("NCpi0 selection")
plt.yticks([0,1,2,3])#fixme
plt.xlabel("true bin")
plt.ylabel("reco bin")
plt.colorbar()
#
plt.show()

In [ ]:
#ccsm = a
#ncsm = a
#ccsm = b
#ncsm = b

trueCCsel = ccsm*recoCCsel
trueCCsel = np.sum(trueCCsel,axis=1)
print(trueCCsel)

trueNCsel = ncsm*recoNCsel
trueNCsel = np.sum(trueNCsel,axis=1)
print(trueNCsel)

In [ ]:
#cheating
#trueCCsel = truetrueCCsel
#trueNCsel = truetrueNCsel

In [ ]:
print('truetrueCCsel =',truetrueCCsel)
print('truetrueNCsel =',truetrueNCsel)
print('recoCCsel =',recoCCsel)
print('recoNCsel =',recoNCsel)
print('trueCCsel =',trueCCsel)
print('trueNCsel =',trueNCsel)
print('effCC =',effCC)
print('effNC =',effNC)
print('fracCC =',fracCC)
print('fracNC =',fracNC)
print('trueratio =',trueratio)

In [ ]:
den = effNC*((1-fracNC) * trueNCsel - fracNC * trueCCsel)
print("den =",den)
num = effCC*((1-fracCC) * trueCCsel - fracCC * trueNCsel)
print("num =",num)
R = num / den
print("R =",R)
#dR2 = (dx*dR/dx)^2 + (dy*dR/dy)^2
tmpx = trueNCsel
tmpy = trueCCsel
tmpa = (1-fracCC)/fracCC
tmpb = fracNC*effNC
tmpc = (1-fracNC)/fracNC
tmpd = fracCC*effCC
dR2 = trueNCsel * ( (tmpd*tmpy*(1-tmpa*tmpc))/(tmpb*(tmpy-tmpc*tmpx)*(tmpy-tmpc*tmpx)) )**2 +\
      trueCCsel * ( (tmpd*tmpx*(tmpa*tmpc-1))/(tmpb*(tmpy-tmpc*tmpx)*(tmpy-tmpc*tmpx)) )**2
dR = np.sqrt(dR2)
print("dR =",dR)


In [ ]:
plt.figure(figsize=(6, 6))
print(bin_edges)
print(bin_edges[1:])
print(bin_edges[:-1])
print(bin_edges[:-1]+0.5*(bin_edges[1:]-bin_edges[:-1]))
bin_centers = bin_edges[:-1]+0.5*(bin_edges[1:]-bin_edges[:-1])
plt.plot(bin_centers, trueratio, 'ro')
plt.errorbar(bin_centers,trueratio,yerr=None,xerr=0.5*(bin_edges[1:]-bin_edges[:-1]),fmt='rd',label='truth')
plt.errorbar(bin_centers,R,yerr=dR,xerr=0.5*(bin_edges[1:]-bin_edges[:-1]),fmt='bo',label='closure')
plt.xlim(bin_edges[0],bin_edges[-1])
plt.ylim(0,2)
plt.ylabel('NC/CC ratio')
plt.xlabel(LABELT)
plt.legend()